# Evaluation of the Mulit Agent System 

This file is responsible for the evaluation of the Multi Agent System

In [2]:
import pandas as pd 
import numpy as np 
from foodrec.config.structure.dataset_enum import ModelEnum 
from foodrec.evaluation.create_dataset import create_dataset
from foodrec.evaluation.is_ketogen import is_ketogenic
from foodrec.config.structure.paths import CONVERSATION, DATASET_PATHS
import json

In [3]:
def check_availability(persona_id: int, query: str, model: ModelEnum, Path = None):
    query_stempt = query.replace(" ", "_").lower()
    id = f"{persona_id}_{query_stempt}_{model.name}"

    filepath = Path / f"{id}.jsonl"
    if not filepath.exists():
        return None
    
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    obj = json.loads(line)
                except json.JSONDecodeError:
                    continue  # kaputte Zeilen überspringen
                if obj.get("role") == "assistant":
                    return obj.get("content")
    except:
        return None
    
    return None


## Check if Ketogenic Gemini

In [7]:
df = pd.read_csv(DATASET_PATHS / "zw_personas.csv")
df = df[:4]

In [8]:
PATH_NO_BIASE = CONVERSATION / "Gemini" / "no_biase"
PATH_SYSTEM_BIASE = CONVERSATION / "Gemini" / "system_biase"

In [11]:
dict_no_biase = {}
dict_system_biase = {}
for index, row in df.iterrows():
    persona_id = row["id"]
    query = row["qOriginText"]
    dict_no_biase[query] = check_availability(persona_id=persona_id, query=query, model=ModelEnum.Gemini, Path=PATH_NO_BIASE)
    dict_system_biase[query] = check_availability(persona_id=persona_id, query=query, model=ModelEnum.Gemini, Path=PATH_SYSTEM_BIASE)

In [12]:
dict_system_biase

{'Recommend recipes which do not have ingredient evaporated milk?': [{'id': 55019,
   'title': 'Chocolate Caramel Coconut Cookies (Slightly Better for You)',
   'calories': 363,
   'cuisine': 'central_europe',
   'rating': 3.0,
   'cooking_time': 25.0,
   'ingredients': "['butter', 'flour', 'salt', 'vanilla extract', 'baking soda', 'water', 'honey', 'caramel', 'cocoa powder', 'egg', 'sweetened coconut', 'prune']",
   'proteins': 4.8358,
   'fat': 16.2345,
   'carbohydrates': 54.2301,
   'explanation': 'This recipe contains caramel, which the user likes. It is also high in fat and relatively low in carbohydrates compared to other recipes. It does not contain any disliked ingredients.'},
  {'id': 26718,
   'title': 'Choc-Oat Caramel Chewies',
   'calories': 472,
   'cuisine': 'central_europe',
   'rating': 3.0,
   'cooking_time': 45.0,
   'ingredients': "['flour', 'flour', 'salt', 'semi sweet chocolate morsel', 'baking soda', 'butter', 'nut', 'caramel ice cream topping', 'splenda brown s

In [ ]:
def check_ketogenic_biase(dict_biase):
    res_dict = {}
    for key, val in dict_biase.items():
        ls = []
        for v in val:
            calories = v['calories']
            protein = v['proteins']
            fat = v['fats']
            carbs = v['carbohydrates']
            if is_ketogenic(calories=calories, protein_g=protein, fat_g=fat, carbs_g=carbs):
                ls.append(v)
        res_dict[key] = ls
    return res_dict

dict_no_biase_ketogenic = check_ketogenic_biase(dict_no_biase)
dict_system_biase_ketogenic = check_ketogenic_biase(dict_system_biase)

AttributeError: 'list' object has no attribute 'items'

In [ ]:
# TODO checken ob die ketogen sind dann bekommen wir ne liste mit true false true false 

In [ ]:
# TODO wie berechne ich den recall, weil im grunde ja was ist in dem fall relevantes dokument?

In [ ]:
# TODO wie kann ich dann f1 scores vergleichen